In [3]:
import os
import sys
import tvm
from tvm import autotvm
sys.path.append("..")
sys.path.append(".")

logdir = "log_sche"
if not os.path.exists(logdir):
    os.mkdir(logdir)

In [5]:
from test_dense import *

ss = [0.2, 0.4, 0.6, 0.8]

channels, group = regnet_parameters("008")
for i in reversed(range(4)):
    width = widths[i]
    channel = channels[i]
    n_trial = 300 if not DEBUG else 10  # no refreshing, so 4 processes are cooperating

    logpath = logdir + f"/c{channel}_w{width}_g{group}"
    if not os.path.exists(logpath):
        os.mkdir(logpath)
    
    bs = TVMDynamicBlockEvaluator(channel, width, group, f"log_dense/c{channel}_w{width}_g{group}", n_trial=n_trial)
    bs.setup()

    # print schedule to log_sche
    for name, layer in bs.layers.items():
        with autotvm.apply_history_best(layer.save_path):
            with tvm.target.Target(layer.target):
                s, arg_bufs = layer._schedule
                layer.func = tvm.build(s, arg_bufs)
                with open(logpath + f"/{name}.txt", "w") as f:
                    f.write(tvm.lower(s, arg_bufs, simple_mode=True).__repr__())

    factors = get_factors(width)
    for g in factors:
        for s in ss:
            sl = int(s * ((width // g) ** 2))
            if sl == 0:
                continue
            logpath_sparse = logpath + f"/l{sl}_m{g}"
            if not os.path.exists(logpath_sparse):
                os.mkdir(logpath_sparse)

            bs.setup_sparse(sl, g, True, True, True)
            # print sparse schedules' to log_sche
            for name, layer in bs.sparse_layers.items():
                with autotvm.apply_history_best(layer.save_path):
                    with tvm.target.Target(layer.target):
                        s, arg_bufs = layer._schedule
                        layer.func = tvm.build(s, arg_bufs)
                        with open(logpath_sparse + f"/{name}.txt", "w") as f:
                            f.write(tvm.lower(s, arg_bufs, simple_mode=True).__repr__())